# Assignment 03: Regression, Learning Curves and Regularization

**Due Date:** Friday 10/04/2024


**Please fill these in before submitting, just in case I accidentally mix up file names while grading**:

Name: Joe Student

CWID-5: (Last 5 digits of cwid)

## Introduction 

In this exercise we will be using what you have learned about linear regression, polynomial regression and
regularization, to explore an artificial dataset.

I have generated a secret dataset.  The dataset uses a polynomial combination of a single parameter.
The unknown function is no less than degree 2, but no more than a degree 20 polynomial.  And some random
noise has also been added into the function, so that fitting it is not a completely trivial or obvious
exercise.  Since the dataset is generated from a polynomial function, the output labels `y` are
real valued numbers.  And thus you will be performing a regression fitting task in this assignment.

Your task, should you choose to accept it, is to load and explore the data from this function.  Your ultimate
goal is to try your best to determine the degree of the polynomial used, and the values of the parameters
then used in the secret function.  Because of the noise added to the data you are given, you will not be able
to exactly recover the parameters used to generate the artificial data.  You will even find that determining the
exact degree of the generating polynomial function is not possible.  How you apply polynomial fitting and 
regularization techniques can give different and better or worse approximations of the true underlying function.

In the below cells, I give instructions for the tasks you should attempt.  You will need to load the data and
visualize it to begin with.  Then you will be asked to apply polynomial fitting and regularization in an attempt
to fit the data.  But ultimately, at the end, you should keep in mind that there is a true function of some
unknown polynomial degree with some coefficient settings used.  You will later be able to see what the
true function was and compare your model performance to the true function you are exploring in this
exercise.

In [1]:
# The following ipython magic will reload changed file/modules.
# So when editing function in source code modules, you should
# be able to just rerun the cell, not restart the whole kernel.
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# By convention, we often just import the specific classes/functions
# from scikit-learn we will need to train a model and perform prediction.
# Here we include all of the classes and functions you should need for this
# assignment from the sklearn library, but there could be other methods you might
# want to try or would be useful to the way you approach the problem, so feel free
# to import others you might need or want to try
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [3]:
# Import functions/moduls from this project.  We manually set the
# PYTHONPATH to append the location to search for this assignments
# functions to just ensure the imports are found
import sys
sys.path.append("../src")

# assignment function imports for doctests and github autograding
# these are required for assignment autograding
from assg_utils import run_unittests, run_doctests
from assg_tasks import task1_load_data, task2_underfit_model, learning_curve_errors, task4_overfit_model, task5_lasso_model, task6_ridge_model

In [4]:
# notebook wide settings to make plots more readable and visually better to understand
np.set_printoptions(suppress=True)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('figure', titlesize=18)
plt.rc('legend', fontsize=14)
plt.rcParams['figure.figsize'] = (12.0, 8.0) # default figure size if not specified in plot
plt.style.use('seaborn-v0_8-darkgrid')

## Task 1: Load Data, Explore and Visualize
------------

You have been given a set of 100 (artificial) data points in the file named `assg-03-data.csv` in our 
data subdirectory.  Start by loading this file into a pandas dataframe.  Explore the data a bit.
Use the `describe()` function to get a sense of the number of values (there should be `m=100` samples),
and their mean and variance.  There are 2 columns, where `x` is the feature, and `y` is the function
value.  Or in other words 'y' is the label we will use for the regression fitting task.  What is the range of
the `x` features?  What is the range of the `y` output label here?


In [5]:
# load data and describe and explore a bit here before doing first task


For this assignment you will need to create a function that loads this data and creates two numpy arrays.
Write/complete the function in `src/assg_tasks.py` named `task1_load_data()`.  This function should
simply load the data from the file, as you did above.  It should extract the `x` features into
a numpy array.  This array needs to be a 2-d column array (so it should be shaped (100,1)).  Likewise
you should extract the regression labels into a numpy array, though the target regression labels should
be returned as a 1-d vector with 100 elements in it.  Your `x` and `y` arrays should be returned
from this function as a result of calling it.

Do not modify or remove the following cell.  When your function correctly loads and returns the
expected feature array `x` and target labels `y`, the following cell should run and pass
the tests for your work.

In [6]:
# do not remove or modify the code in this cell
x, y = task1_load_data()
run_unittests(['test_task1_load_data'])

test_loaded_types (test_assg_tasks.test_task1_load_data.test_loaded_types)
test_loaded_types ... ok
test_x_properties (test_assg_tasks.test_task1_load_data.test_x_properties)
test_x_properties ... FAIL
test_y_properties (test_assg_tasks.test_task1_load_data.test_y_properties)
test_y_properties ... FAIL

FAIL: test_x_properties (test_assg_tasks.test_task1_load_data.test_x_properties)
test_x_properties
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/defer.py", line 209, in maybeDeferred
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 227, in runWithWarningsSuppressed
    raise exc_info[1].with_traceback(exc_info[2])
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 223, in runWithWarningsSuppressed
    result = f(*a, **kw)
  File "/workspaces/assg03/notebooks/../src/test_a

<unittest.runner.TextTestResult run=3 errors=0 failures=2>

Finally before moving on, create a scatter plot of the data that you are using in this assignment
in the following cell.  Make sure you are using a scatter plot, since this is raw data and
not a model.  You should label your axis on the plot.

In [7]:
# visualize the data we loaded here

## Task 2: Create an Underfit Model

The degree of the secret function is more than 2, but probably less than 20 in this assignment.  Lets create a model
that should underfit the data.  You will create a degree 2 (quadratic) model.  Underfit a degree 2 polynomial regression to the
data.  

You should complete the function in `src/asst_tasks.py` named `task2_underfit_model()` before
running the next cell to test your work.  This function takes the `x` input features and
`y` target labels as input, which should have been created successfully in the previous task.

Do the following to create a model
- Create a `scikit-learn` pipeline.
- The first step in the pipeline should be a `PolynomialFeatures` transformer that transforms the input features
  to a degree 2 polynomial.  You should not generate the bias terms from this transformer, as the linear regression
  model you will use does not expect a dummy input feature in the input feature data.
- Then for the second step have a standard `scikit-learn` `LinearRegression` model.  You should not specify any
  meta parameters for this underfit model, use all of the `scikit-learn` default settings for a linear regression.

Do not modify or remove the following code cell.  If you create your underfit model pipeline correctly, the
tests of your work in the next cell should all pass.

In [8]:
# do not remove or modify the code in this cell
underfit_model = task2_underfit_model(x, y)
run_unittests(['test_task2_underfit_model'])

test_coef (test_assg_tasks.test_task2_underfit_model.test_coef)
test_coef ... ERROR
test_intercept (test_assg_tasks.test_task2_underfit_model.test_intercept)
test_intercept ... ERROR
test_model_pipeline (test_assg_tasks.test_task2_underfit_model.test_model_pipeline)
test_model_pipeline ... FAIL
test_r2score (test_assg_tasks.test_task2_underfit_model.test_r2score)
test_r2score ... ERROR
test_rmse (test_assg_tasks.test_task2_underfit_model.test_rmse)
test_rmse ... ERROR

ERROR: test_coef (test_assg_tasks.test_task2_underfit_model.test_coef)
test_coef
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/defer.py", line 209, in maybeDeferred
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 227, in runWithWarningsSuppressed
    raise exc_info[1].with_traceback(exc_info[2])
  File "/opt/conda/lib/python3.12/site-

<unittest.runner.TextTestResult run=5 errors=4 failures=1>

In the next cell, use introspection of your fitted (pipeline) model to display the intercept and fitted
coeffecients of the model.  Also determine the final RMSE and $R^2$ score of the fit.  Display them
in the next cell(s) for future reference and discussion.

You should get a single intercept parameter.  Since you are underfitting a degree 2 model here, you should
expect 2 coefficients, one for the $x^1$ original feature term, and another for the generated
$x^2$ square term for your model.

Make note of the $R^2$ score, and especially the RMSE fit of your model here.  Do you expect the
undefit model to be a good model or a bad model of this data?

In [9]:
# display asked for model parameters and scores here


## Task 3: Calculate and Display Learning Curves

Before moving on we need to evaluate how well the model is performing.  To do this you are first going to write a function
that will calculate train and test errors of your model, so that we can plot the learning curves of this and all of the other
models you build in this assignment.

Write the function named `learning_curve_errors()`. This function is in the `src/assg_tasks.py` file of this
assignment. 

The function takes a model (pipeline) and the `x` input features and `y` regression labels as input.  To calculate
the learning curve errors, you first need to split the data using an 80/20 train/test split.  Then you are going
to train the model with the training data.  You will first train the modle using only 1 of the training inputs.
The model will be evaluated on the data point it was trained with, and then on the test data.  Then you will train
a model with 2 data points and calculate the errors, and so on until you have trained a model on all of the
training data.  For each model from 1, 2 up to all of the training data, you will calculate the RMSE error on
from the model on the data you just trained it with, and also the RMSE error on the 20% test data you held back
at the start.

So to complete this function, perform the following steps in the `learning_curve_errors()` function:

1. Split the `x` `y` data you are given in the function into 80% training data and 20% test data.  Use
   the appropriate `sckit-learn` function to perform this split, don't do it by hand.
2. You will need to return the accumulated train and test errors, so create empty Python lists or numpy
   arrays to accumulate and return the errors as you calculate them.
3. You need a loop that trains/fits a model then calculates the RMSE error on the data trained with and
   on the held back test data.
   - Train with 1, 2, 3, ..., all of the train data in the loop
   - Evaluate the error on the training data (make sure you only evaluate on the data you trained with, not
     all of the training data.
   - Evaluate the error on the 20% held back test data.
   - Append both of these to the array/list you accumulate of the train and test errors
4. This function should return the calculated lists of training errors and test errors.

You should not remove or modify the following code cell(s), which will test that your implementation of
the `learning_curve_errors()` function is working as expected.

In [10]:
# do not remove or modify the code in this cell
run_unittests(['test_learning_curve_errors'])
#run_doctests(learning_curve_errors, globals())

test_test_errors (test_assg_tasks.test_learning_curve_errors.test_test_errors)
test_test_errors ... FAIL
test_train_errors (test_assg_tasks.test_learning_curve_errors.test_train_errors)
test_train_errors ... FAIL

FAIL: test_test_errors (test_assg_tasks.test_learning_curve_errors.test_test_errors)
test_test_errors
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/defer.py", line 209, in maybeDeferred
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 227, in runWithWarningsSuppressed
    raise exc_info[1].with_traceback(exc_info[2])
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 223, in runWithWarningsSuppressed
    result = f(*a, **kw)
  File "/workspaces/assg03/notebooks/../src/test_assg_tasks.py", line 80, in test_test_errors
    self.assertEqual(len(self.test_errors), 

<unittest.runner.TextTestResult run=2 errors=0 failures=2>

Use this function to plot the learning curves for your underfit model on all of the
data you have.

1. You should call your `learning_curve_errors()` function on your underfit model, and the `x` inputs and `y` labels.
   You will need to keep the returned train and test errors.
2. Before calling the function to generate the learning curve errors, set the NumPy random seed to 42.  If you do this
   you will always get the same train/validation split of data when the function is called.  If you do this you
   should see that performance converges for train and validation RMSE after using around only 10 data points or so.
3. Plot your train and test errors on the same figure.  Label the axis appropriately, and use a legend in your figure
   to identify which curve is the training error and which is the testing error.  You should use lines for both
   of these curves on this figure.  Make sure you cn see where these converge on the figure, you may need to limit the
   y axis to be able to see and read this from your figure.   

In [11]:
# call your learning_curve_errors() function to the the test and train errors


In [12]:
# plot the resulting learning curve


You may want or need to limit the y axis to better view the learning curves.  You should see a
bit of a gap usually (because of random split in the learning curves function, results will
vary each time you run).  But usually with a training set size of 20 points or so you should find that
results converge.  Note the RMSE that you are getting on the training and validation data once they
converge on the underfit model.

Recall from our discussion in class, that the RMSE obtained on an underfit model does give you some information.
We would hope that if we can find a model with the right power, that the validation results will still converge
relatively quickly, but with better overall RMSE performance than the underfit model can obtain.

## Task 4: Create an Overfit Model

The next step when trying to determine what we can achieve with a type of ML model is to try and
overfit the data with a too powerful model.

Repeat the previous steps, but with a degree 100 polynomial.  This should be many times larger than the
actual degree of the polynomial used in the secret function for this assignment.

Do all of the previous steps again for a degree 100 model in the following cells.

1. Implement the function named `task4_overfit_model()` in the `src/assg_tasks.py` file.  In this function you need to:
   - Create a pipeline to generate degree 100 features that feeds into a default `LinearRegression` model.
   - Fit the model to all of the data given as input to the function.
   - Return the trained overfit model for testing.

Do not modify or remove the following code cell.  If your model performs as expected, you should
pass the tests performed on your model in the next cell.


In [13]:
# do not remove or modify the code in this cell
overfit_model = task4_overfit_model(x, y)
run_unittests(['test_task4_overfit_model']);

test_coef (test_assg_tasks.test_task4_overfit_model.test_coef)
test_coef ... ERROR
test_intercept (test_assg_tasks.test_task4_overfit_model.test_intercept)
test_intercept ... ERROR
test_model_pipeline (test_assg_tasks.test_task4_overfit_model.test_model_pipeline)
test_model_pipeline ... FAIL
test_r2score (test_assg_tasks.test_task4_overfit_model.test_r2score)
test_r2score ... ERROR
test_rmse (test_assg_tasks.test_task4_overfit_model.test_rmse)
test_rmse ... ERROR

ERROR: test_coef (test_assg_tasks.test_task4_overfit_model.test_coef)
test_coef
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/defer.py", line 209, in maybeDeferred
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 227, in runWithWarningsSuppressed
    raise exc_info[1].with_traceback(exc_info[2])
  File "/opt/conda/lib/python3.12/site-packag

Once your overfit model is working, do the following

1. Report the intercept and slope coefficients for the model.  There should be consideradbly more coefficients this
   time around for your overfit model.
2. Report $R^2$ and RMSE scores obtained by this overfit model.
3. Plot the learning curves of the overfit model.  You should reuse your task 2 function to generate the
   learning curve errors to be plotted.  Make sure you set the random seed to 42 again before generating
   the learning curve errors.  Also make sure you limit the y axis if needed to be able to see where
   if the model converges and what error level the train error settles at.

You may want to stop and compare the results of the underfit and overfit model.  What od you notice about
the $R^2$ score?  What about the RMSE error reported?  What do you notice about the coefficient values
that were fit for the two models?

In [14]:
# report the fitted model parameters


In [15]:
# finally generate the learning curves for the overfit model.  Set the
# random seed to 42 before plotting learning curves again.  And
# limit y axis if needed to be able to see where curves converge


In [16]:
# plot the resulting learning curve


## Task 5: Estimate True Function Degree using Lasso Regularization

The Lasso Regularization that we discussed in class (using absolute value for the regularization term,
also known as $\ell_1$ regularization), has the effect of driving unneeded parameters to 0.  So in our current
assignment, it can give us some insight into the true degree of the secret function polynomial used to
generate the data you were given.

In the following cell, use Lasso regularization on the degree 100 model again.  Try exploring values
of alpha used in your Lasso regularization model.  Your goal is to find a value of alpha
that will achieve an RMSE on your fitted model much better than the underfit RMSE and hopefully approaching the overfit
model error.  You probably won't be able to get quite the same RMSE, but you should be able to
find values of `alpha` that are much better than the underfit model achieves.

Do all of the previous steps again for a Lasso regularization model in the following cells.  As before, your actual code needs to
go into the `task5_lasso_model()` function in the `src/assg_tasks.py` file.

1. Create a pipeline to generate degree 100 features for a `Lasso` regression model.
2. Fit the Lasso regularized pipeline model to all of the data, exploring good values of `alpha` to use for the regularization.

Do not remove or modify the following code cell line.  It will test that you create a `Lasso` model
as expected, and that it gets a reasonable fit on the data.  Your model will be tested to see that
it meets reasonable expectations for this task by the unit tests.


In [17]:
# do not remove or modify the code in this cell
lasso_model = task5_lasso_model(x, y)
run_unittests(['test_task5_lasso_model']);

test_coef (test_assg_tasks.test_task5_lasso_model.test_coef)
test_coef ... ERROR
test_model_pipeline (test_assg_tasks.test_task5_lasso_model.test_model_pipeline)
test_model_pipeline ... FAIL
test_r2score (test_assg_tasks.test_task5_lasso_model.test_r2score)
test_r2score ... ERROR
test_rmse (test_assg_tasks.test_task5_lasso_model.test_rmse)
test_rmse ... ERROR

ERROR: test_coef (test_assg_tasks.test_task5_lasso_model.test_coef)
test_coef
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/defer.py", line 209, in maybeDeferred
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 227, in runWithWarningsSuppressed
    raise exc_info[1].with_traceback(exc_info[2])
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 223, in runWithWarningsSuppressed
    result = f(*a, **kw)
  File "/work

Then as before, do the following with the returned fitted `Lasso` model:

1. Report the intercept and slope coefficients for the model. Note which parameters have been driven to 0 by the
   Lasso regularization.
2. Report $R^2$ and RMSE scores obtained by this Lasso regularized model.
3. Plot the learning curves of the Lasso regularized model.  Reuse your `learning_curve_errors()` function.
   Make sure that you set the random seed to 42 before calculating the learning curve errors, and that
   you plot the learning curves the same as before so that it is apparent if the model is underfitting,
   overfitting or converging.

In [18]:
# report the model coefficients and scores for the lasso regularization model


In [19]:
# finally generate the learning curve errors for the lasso regularization
# model.  Set the random seed to 42 before plotting learning curves
# again.  And limit y axis if needed to be able to see where curves
# converge.  You will probably get ConvergenceWarnings here, though you
# may want to and probably should increase the `max_iter` for you model
# to reduce these warnings.


In [20]:
# plot the resulting learning curve


You will know you are getting somewhere in the appropriate neighborhood for good
values of the `alpha` metaparameter when:

1. Your R^2 and RMSE scores are similar to the overfit model, and are certainly better than the underfit model.
2. The learning curves should "converge" again here, similar to the underfit model.  Probably sometime after using
   20 or 30 data points in the training set size.  But they should converge to an RMSE lower than that seen for
   the underfit model.

## Task 6: Use Ridge regularization

Try and at least match, if not improve upon, the best RMSE you can find from Lasso regularization using
Ridge regularization instead.  You should find that coefficients will not mostly be driven to 0's like
before for the Lasso regularization. But you may be more easily able to find an alpha that gets an improvement
in terms of RMSE over what you can obtain with the Lasso model.

Again perform all of the following for a Ridge regularization model, finding an appropriate value of the `alpha`
meta parameter.  The function where you implement your Ridge model is in the `task6_ridge_model()` function
in the `src/assg_tasks.py` file.

1. Create a pipeline to generate degree 100 features for a `Ridge` regression model.
2. Fit the Ridge regularized model pipeline to all of the data, exploring good values of `alpha` to use for the regularization.
3. Report the intercept and slope coefficients for the model. Note that most parameters are not 0 for this type of regularization.
5. Report $R^2$ and RMSE scores obtained by this Ridge regularized model.
6. Plot the learning curves of the Ridge regularized model.

As usual make sure you do not remove or change the following cell that tests your creating of the
Ridge model.

In [21]:
# do not remove or modify the code in this cell
ridge_model = task6_ridge_model(x, y)
run_unittests(['test_task6_ridge_model']);

test_coef (test_assg_tasks.test_task6_ridge_model.test_coef)
test_coef ... ERROR
test_model_pipeline (test_assg_tasks.test_task6_ridge_model.test_model_pipeline)
test_model_pipeline ... FAIL
test_r2score (test_assg_tasks.test_task6_ridge_model.test_r2score)
test_r2score ... ERROR
test_rmse (test_assg_tasks.test_task6_ridge_model.test_rmse)
test_rmse ... ERROR

ERROR: test_coef (test_assg_tasks.test_task6_ridge_model.test_coef)
test_coef
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/defer.py", line 209, in maybeDeferred
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 227, in runWithWarningsSuppressed
    raise exc_info[1].with_traceback(exc_info[2])
  File "/opt/conda/lib/python3.12/site-packages/twisted/internet/utils.py", line 223, in runWithWarningsSuppressed
    result = f(*a, **kw)
  File "/work

In [22]:
# report model parameters and scores for the lasso regularization model


In [23]:
# finally generate the learning curves for the lasso regularization
# model.  Set the random seed to 42 before plotting learning curves
# again.  And limit y axis if needed to be able to see where curves
# converge


In [24]:
# plot the resulting learning curve


## Task 7: Summary and Discussion of Findings

Fill out the following table with the values you obtain in your work on the assignment.

| Model    | `alpha` | $R^2$  | RMSE   |
|----------|---------|--------|--------|
| underfit | none    | 0.0000 | 0.0000 | 
| overfit  | none    | 0.0000 | 0.0000 |
| Lasso    | 0.0000  | 0.0000 | 0.0000 |
| Lasso    | 1.0000  | 0.0000 | 0.0000 |
| Lasso    | 2.0000  | 0.0000 | 0.0000 |
| Ridge    | 0.0000  | 0.0000 | 0.0000 |
| Ridge    | 1.0000  | 0.0000 | 0.0000 |
| Ridge    | 2.0000  | 0.0000 | 0.0000 |

I would like to see at least 3 different values each for your Lasso and Ridge models.  

For the first Lasso regularization model, try and find an `alpha` that gets the lowest
RMSE that you can, and also drives as many of the coefficients to 0 as you can.  This can give
you an approximate estimate of the degree of the secret polynomial function you are trying to model.
Show a value that is larger than this `alpha` (more regularization), that drives even more parameters
to 0, but probably increases the RMSE error.  Also show a value with smaller `alpha`, that probably
is overfitting a bit, so you get the same RMSE, but maybe many or all of the coefficients are still non zero.

For the Ridge regression, for the secret function given in this assignment you will probably find that you
can get better RMSE than you can for LASSO.  Again try and find the value for the `alpha` amount of
regularization to Ridge that gives the minimum RMSE you can get, but doesn't seem to be overfitting.  Report
on a value of `alpha` bigger than this (more regularization) that seems to start to underfit so that the RMSE
goes up.  And also on a value smaller than the best you can find, so that you may be overfitting.

Once you have filled out the above table, please discuss your observations about the underfit, overfit,
Lasso and Ridge models that you have obsreved.  Your discussion must be in this markdown cell, in the
section marked `Discussion` below.  You should make observations on what the coefficients look like for the 
various models you fit.  You should give an estimate from your Lasso regression on what the degree of the
polynomial seems to be from your testing of alpha to find a model with the lowest RMSE that drives as many
coefficients to 0 as possible.  


### Discussion

Place your discussion here.


## Task 8: More Systematic Grid Meta-Parameter Search

While it is possible (and useful) to get a feel for what meta-parameter values you need to obtain
good results from regularization and different models, at times we need to be more systematic.

You have seen some examples of using the Scikit-learn grid search to more systematically
explore model meta parameters.  The following is a tutorial specifically for beginners
on using [scikit-learn GridSearchCV for Beginners](https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee)
that you might find helpeful to look at as well when doing this section.

In the following, perform a more systematic grid search of the alpha parameter for a Lasso
regression using a degree 100 polynomial as input.  Report the RMSE performance of
the best model you find.  Make sure you search a fairly good range of `alpha` parameters.
Also report the intercept and coefficients of the model.

Did the best model have a similar `alpha` value that you got trying to tune it by hand?  Do the
parameters that were driven to 0 look similar or different from the value of `alpha` you found
previously?  For extra consideration, create a plot to visualize the RMSE performance for the range of
`alpha` values that you explored in the grid search.

In [25]:
# create grid search of Lasso/l1 over alpha parameter space


In [26]:
# fit the model to perform the grid search


In [27]:
# display the best parameter you found


In [28]:
# report intercept and slope coefficients for the lasso regularization model


In [29]:
# report R^2 and RMSE performance of the lasso regularization model on all of the data here


In [30]:
# visualize performance as a function of the alpha regularization parameter


As a final activity for this assignment, below you have been given an example of a setup for a
`GridSearchCV` that will instead search all polynomial degrees from 2 to 100.  Modify the following 
code if necessary to run in your assignment (may need to change name of input `x` or output `y` or
other minor tweaks?)  Observe the polynomial degree obtained by this grid search on the data.  Does
that surprise your, or look resonable to some of the estimates you have made about the secret
function?

In [31]:
# a helper function.  This returns a fit/transformer pipeline, as GridSearchCV expects
# but we can call it to generate pipelines with different degree polynomials
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree, include_bias=False), LinearRegression(**kwargs))

# we set up grid search on single degree meta-parameter of the PolynomialFeatures part of the pipeline
# the result is that degrees from 1 to 100 are tried for PolynomialFeatures here
degrees = np.arange(2, 100)
parameters = {'polynomialfeatures__degree': degrees}

# the grid search over the polynomial degrees on standard Linear Regression
poly_gridcv = GridSearchCV(PolynomialRegression(), parameters, scoring='neg_mean_squared_error', cv = 5)

In [32]:
# fit the model to perform the grid search
#poly_gridcv.fit(x, y)

# display the best parameter we found
#print('Grid search best alpha parameter:', poly_gridcv.best_params_)
#print('Grid search best MSE: ', poly_gridcv.best_score_)
#print('Grid search best RMSE: ', np.sqrt(-poly_gridcv.best_score_))

In [33]:
# report intercept and slope coefficients for the lasso regularization model
#print('Intercept   : ', poly_gridcv.best_estimator_['linearregression'].intercept_)
#print('Coefficients: ', poly_gridcv.best_estimator_['linearregression'].coef_)